In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold, learning_curve, GridSearchCV, RandomizedSearchCV

In [2]:
# importing dataset
from google.colab import files
uploaded = files.upload()

Saving ML-CUP23-TR.csv to ML-CUP23-TR.csv


In [3]:
# importing dataset
from google.colab import files
uploaded1 = files.upload()

Saving ML-CUP23-TS.csv to ML-CUP23-TS.csv


ho eliminato le prime 7 righe dai dataset contenenti informazioni come mail, ecc.

In [4]:
import io
df_traincup = pd.read_csv(io.BytesIO(uploaded['ML-CUP23-TR.csv']), names=['id', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'target_x', 'target_y', 'target_z'], delimiter=',')

In [5]:
import io
df_testcup = pd.read_csv(io.BytesIO(uploaded1['ML-CUP23-TS.csv']), names=['id', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'target_x', 'target_y', 'target_z'], delimiter=',')

In [6]:
df_traincup

,id,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,target_x,target_y,target_z
0,1,-0.917280,-0.712727,-0.989904,0.992819,0.993649,0.995543,0.711074,0.407645,-0.688548,0.616890,7.897453,-35.936382,21.077147
1,2,-0.858784,0.998755,-0.998396,0.999909,0.316503,-0.951897,-0.163139,0.980982,0.661759,-0.800155,-9.330632,19.901571,6.069154
2,3,-0.990441,0.958726,-0.998675,0.997216,0.987166,0.356483,-0.279689,0.599163,-0.684630,0.922901,14.849400,3.374090,19.667479
3,4,0.937117,0.984474,-0.612420,0.999812,0.728623,-0.539962,-0.165939,0.999352,-0.921444,-0.974766,-46.591854,13.734777,17.953600
4,5,-0.906628,-0.884567,-0.932487,0.941037,0.978134,0.998179,0.749606,-0.590599,-0.508268,0.691798,8.217500,-45.885254,14.894251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,-0.803560,-0.878859,-0.978391,0.957539,0.996655,0.996924,-0.226305,0.097814,-0.922666,0.934153,15.389553,-41.068806,27.513502
996,997,0.718841,0.995748,-0.942678,0.999929,0.747763,-0.808726,0.176132,0.999100,-0.753970,-0.969009,-36.228770,13.067430,11.672133
997,998,-0.977912,-0.971108,0.956233,-0.979524,0.222033,0.986609,0.658273,-0.987310,0.937697,0.143420,7.265506,-53.497242,2.815666
998,999,0.284803,-0.988684,-0.427197,0.883317,0.993302,0.999500,-0.019456,-0.648110,-0.955231,0.901298,5.545274,-63.348396,27.989340


In [7]:
def split_data(data: pd.DataFrame, cols_name_split: list, rows_split_perc=1):
  '''
      It makes the split of the columns passed in \"cols_name_split\" and the split of the rows based on the \
      percentage \"rows_split_perc\".\n
      If \"cols_name_split\" = [] -> the method returns: (data, None).\n
      else -> (data_splitted_x, data_splitted_y) with rows_split_perc=1 \
          or (data_splitted_x_train, data_splitted_y_train, data_splitted_x_val, data_splitted_y_val) with rows_split_perc!=1.\n
      So this method makes split on columns or on rows (or both).\n\n
      Returns a tuple of two new DataFrames: (x,y).\n
      - x: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y: are the columns indentified by the list \"cols_name_split\".\n
      or a tuple of this format (x_train, y_train, x_val, y_val) with:
      - x_train: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y_train: are the columns indentified by the list \"cols_name_split\" used for Training.\n
      - x_val: is like \"df\" without the columns specified in the list \"cols_name_split\".\n
      - y_val: are the columns indentified by the list \"cols_name_split\" used for Validation.\n\n
      The parameters are:\n
      - df: the input DataFrame.\n
      - cols_name_split: list of names of target columns.\n
      - rows_split_perc: percentage of the data to split for Training, and so the 1-rows_split_perc percentage for Validation.
  '''
  # Case of no columns split
  if cols_name_split == []:
      # Case of rows split
      if rows_split_perc != 1:
          return np.split(data, [int(data.shape[0] * rows_split_perc)], axis=0)
      # Case of no rows and no columns split
      else:
          raise ValueError

  # Columns split
  y = data[cols_name_split].copy(deep=True)
  x = data.drop(columns=cols_name_split,axis=1).copy(deep=True)

  # Case of only columns split
  if rows_split_perc == 1:
      return x, y

  # Case of both splits (rows split and columns split)
  else:
      #x_train = x[:int(x.shape[0] * rows_split_perc), :]
      #x_val = x[int(x.shape[0] * rows_split_perc):, :]
      x_train, x_val = np.split(x, [int(x.shape[0] * rows_split_perc)], axis=0)
      #y_train = y[:int(x.shape[0] * rows_split_perc), :]
      #y_val = y[int(x.shape[0] * rows_split_perc):, :]
      y_train, y_val = np.split(y, [int(y.shape[0] * rows_split_perc)], axis=0)
      return x_train, y_train, x_val, y_val

## DATA SPLIT

In [9]:
# Saving the ID columns
df_id_train: pd.DataFrame = df_traincup['id']
df_id_test: pd.DataFrame = df_testcup['id']

# Drop the ID columns
df_train = df_traincup.drop(columns=['id'],axis=1).copy(deep=True)
df_test = df_testcup.drop(columns=['id'],axis=1).copy(deep=True)

# Split of columns and rows (0.8/0.2) into: TR set and Internal TS set
x_train, y_train, x_internal_test, y_internal_test = split_data(
    data=df_train,
    cols_name_split=['target_x','target_y','target_z'],
    rows_split_perc=0.8
)

# Split on columns
x_test, y_test = split_data(data=df_test, cols_name_split=['target_x','target_y','target_z'])

# Print of the shapes
print(f"[IDs TR SET]: " + str(df_id_train.shape))
print(f"[IDs TS SET]: " + str(df_id_test.shape))
print(f"[TR SET - x]: " + str(x_train.shape))
print(f"[TR SET - y]: " + str(y_train.shape))
print(f"[Internal TS SET - x]: " + str(x_internal_test.shape))
print(f"[Internal TS SET - y]: " + str(y_internal_test.shape))
print(f"[TS SET - x]: " + str(x_test.shape))
print(f"[TS SET - y]: " + str(y_test.shape))

[IDs TR SET]: (1000,)
[IDs TS SET]: (900,)
[TR SET - x]: (800, 10)
[TR SET - y]: (800, 3)
[Internal TS SET - x]: (200, 10)
[Internal TS SET - y]: (200, 3)
[TS SET - x]: (900, 10)
[TS SET - y]: (900, 3)


In [10]:
# Define custom loss function
def euclidean_distance_loss(y_true, y_pred):
    return np.sqrt(np.sum(np.square(y_pred - y_true)))

# Define a custom scorer based on the custom loss function
def MEE(y_true, y_pred):
    return np.mean(euclidean_distance_loss(y_true, y_pred))

# Use make_scorer to create a scorer suitable for RandomizedSearchCV
scorer = make_scorer(MEE, greater_is_better=False)

### STANDARDIZATION


In regression problems it is customary to normalize the output too, because the scale of output and input features may differ.

If you normalize your data, you will have a cost function which is well behaved. means that you can find local parts more easily. The reason is that you have to construct the output using the input features in regression problems. It is difficult to make large values with small normalized features. Instead with small numbers, making a normalized output, is easier and can be learned faster.



In [23]:
scaler = StandardScaler()

scaler.fit(x_train)
x_train_std = scaler.transform(x_train)
x_train=pd.DataFrame(x_train_std)

In [25]:
scaler.fit(x_internal_test)
x_internal_test_std = scaler.transform(x_internal_test)
x_internal_test = pd.DataFrame(x_internal_test_std)

In [26]:
scaler.fit(y_train)
y_train_std = scaler.transform(y_train)
y_train = pd.DataFrame(y_train_std)

In [27]:
scaler.fit(y_internal_test)
y_internal_test_std = scaler.transform(y_internal_test)
y_internal_test = pd.DataFrame(y_internal_test_std)

In [31]:
y_train

,0,1,2
0,0.723809,-0.728015,0.379053
1,0.061300,1.209682,-1.080945
2,0.991148,0.636143,0.241919
3,-1.371588,0.995681,0.075190
4,0.736117,-1.073262,-0.222428
...,...,...,...
795,0.847542,-1.233534,-1.420234
796,0.602785,-1.690770,0.421536
797,0.607146,-1.825667,-1.351667
798,0.506989,-1.854067,-0.745900


## TRAINING OF 3 UNIVARIATE SVR

In [28]:
from sklearn.svm import SVR

In [29]:
# Parameter grid for the Grid Search
param_grid = {'C': [10], 'epsilon': [0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf'], 'degree':[2], 'gamma': ['scale', 'auto'], 'max_iter': [340]} #no precomputed kernel perché ha bisogno di una matrice quadrata (n,n)

### label x

In [33]:
y_train_x = y_train[0]
y_test_x = y_internal_test[0]

In [34]:
y_train_x

0      0.723809
1      0.061300
2      0.991148
3     -1.371588
4      0.736117
         ...   
795    0.847542
796    0.602785
797    0.607146
798    0.506989
799    0.752538
Name: 0, Length: 800, dtype: float64

In [35]:
grid_SVR_x = GridSearchCV(SVR(),param_grid, verbose=0, cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [36]:
grid_SVR_x.fit(x_train,y_train_x)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10], 'degree': [2],
                         'epsilon': [0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto'], 'kernel': ['rbf'],
                         'max_iter': [340]},
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [61]:
print("Best parameters obtained by Random Search - label x:", grid_SVR_x.best_params_)

Best parameters obtained by Random Search - label x: {'C': 10, 'degree': 2, 'epsilon': 0.0001, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}


In [38]:
best_regressor_x = grid_SVR_x.best_estimator_

In [39]:
y_train_pred_x = best_regressor_x.predict(x_train)

In [40]:
train_loss = MEE(y_train_x, y_train_pred_x)
print("MEE - label x: ", train_loss)

MEE - label x:  2.3408193166228535


In [41]:
# Assuming x_val and y_val are your validation data
y_val_pred_x = best_regressor_x.predict(x_internal_test)
val_loss_x = MEE(y_test_x, y_val_pred_x)
print("val loss - label x: ", val_loss_x)

val loss - label x:  1.3099307917033947


### label y

In [53]:
y_train_y = y_train[1]
y_test_y = y_internal_test[1]

#training
grid_SVR_y = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)
grid_SVR_y.fit(x_train,y_train_y)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10], 'degree': [2],
                         'epsilon': [0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto'], 'kernel': ['rbf'],
                         'max_iter': [340]})

In [54]:
print("Best parameters obtained by Random Search - label y:", random_SVR_y.best_params_)
best_regressor_y = grid_SVR_y.best_estimator_
y_train_pred_y = best_regressor_y.predict(x_train)

#training validation
train_loss_y = MEE(y_train_y, y_train_pred_y)
print("MEE - label y: ", train_loss_y)

Best parameters obtained by Random Search - label y: {'C': 10, 'degree': 2, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}
MEE - label y:  1.2663088261988547


In [55]:
#internal validation
y_val_pred_y = best_regressor_y.predict(x_internal_test)
val_loss_y = MEE(y_test_y, y_val_pred_y)
print("val loss - label y: ", val_loss_y)

val loss - label y:  1.2664074642510657


### label z

In [56]:
y_train_z = y_train[2]
y_test_z = y_internal_test[2]

#training
grid_SVR_z = GridSearchCV(SVR(),param_grid,refit=True,verbose=0,cv=5)
grid_SVR_z.fit(x_train,y_train_z)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=340).  Consider pre-

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [10], 'degree': [2],
                         'epsilon': [0.1, 0.01, 0.001, 0.0001],
                         'gamma': ['scale', 'auto'], 'kernel': ['rbf'],
                         'max_iter': [340]})

In [57]:
print("Best parameters obtained by Random Search - label z:", random_SVR_z.best_params_)
best_regressor_z = grid_SVR_z.best_estimator_
y_train_pred_z = best_regressor_z.predict(x_train)

#training validation
train_loss_z = MEE(y_train_z, y_train_pred_z)
print("MEE - label z: ", train_loss_z)

Best parameters obtained by Random Search - label z: {'C': 10, 'degree': 2, 'epsilon': 0.001, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 340}
MEE - label z:  2.0058014661511963


In [58]:
#internal validation
y_val_pred_z = best_regressor_z.predict(x_internal_test)
val_loss_z = MEE(y_test_z, y_val_pred_z)
print("val loss - label z: ", val_loss_z)

val loss - label z:  1.3699052965970009


### MEE COMBINED

In [59]:
# Calcolo del Mean Euclidean Error combinato
combined_error = np.mean([val_loss_x, val_loss_y, val_loss_z], axis=0)
mean_euclidean_error = np.mean(combined_error)

print("Mean Euclidean Error combinato:", mean_euclidean_error)

Mean Euclidean Error combinato: 1.3154145175171537
